In [38]:
import pandas as pd
from pandas import DataFrame
from pandas import concat
import matplotlib.pyplot as plt
import xgboost as xgb
from sklearn.datasets import load_iris
from sklearn.ensemble import RandomForestClassifier
import pandas as pd
import numpy as np
from sklearn.cross_validation import cross_val_score # K折交叉验证模块
from sklearn.metrics import roc_auc_score
from sklearn.preprocessing import MinMaxScaler
from sklearn import preprocessing
from sklearn import svm
from sklearn.feature_selection import VarianceThreshold
from sklearn.feature_selection import RFE
from keras.models import Sequential
from keras.layers import Dense,Dropout
from keras.wrappers.scikit_learn import KerasClassifier
from keras.utils import np_utils
from numpy import argmax
from xgboost.sklearn import XGBClassifier

%matplotlib inline

In [2]:
def submit(preds, test_feature, label_map):
    submission = test_feature[['id']]
    submission['pred'] = [label_map[l] for l in np.nditer(preds)]
    submission.to_csv('./predicts/0216_1_partition.csv', header=False, index=False)
    return submission
    

In [3]:
test_feature = pd.read_csv('./test_feature_partition0215_p_100_var.csv',header=None)
test_feature.head()
train_feature = pd.read_csv('./train_feature_partition0215_p_100_var.csv',header=None)
test_feature.rename(columns={0:'id'}, inplace=True)
train_feature.head()


,0,1,2,3,4,5,6,7,8,9,...,196,197,198,199,200,201,202,203,204,205
0,696220,1206.6,4833.9,-362.9,2807192.6,2842.6,3028.4,2958.7,1128.8,2927.0,...,2.5,-98.0,2.2,-89.8,1.5,-94.5,2.1,-75.8,2.0,star
1,911466,112.2,166.7,-2.9,899.3,17.0,1.1,20.7,0.3,23.8,...,0.2,89.4,0.4,88.8,0.3,88.5,0.2,77.3,6.9,star
2,915856,61.4,126.5,24.1,434.2,80.8,2.2,86.1,1.4,90.3,...,0.1,33.4,0.1,30.7,0.1,30.9,0.1,31.5,0.0,star
3,720026,12073.5,20181.5,3666.1,17101718.7,13748.7,69732.8,15392.9,43757.4,13870.7,...,47.0,6240.3,92.9,6072.5,116.9,6095.6,28.4,6039.1,60.9,star
4,1198160,379.8,491.7,-225.7,11729.1,-8.1,68.4,8.5,26.5,51.7,...,0.8,392.1,1.4,395.1,0.5,396.7,0.4,367.7,11.0,star


In [4]:
train_feature.shape[1]

206

In [5]:
columns = {0:'id'}
columns[train_feature.shape[1]-1] = 'label'
train_feature.rename(columns=columns, inplace=True)
train_feature.head()

,id,1,2,3,4,5,6,7,8,9,...,196,197,198,199,200,201,202,203,204,label
0,696220,1206.6,4833.9,-362.9,2807192.6,2842.6,3028.4,2958.7,1128.8,2927.0,...,2.5,-98.0,2.2,-89.8,1.5,-94.5,2.1,-75.8,2.0,star
1,911466,112.2,166.7,-2.9,899.3,17.0,1.1,20.7,0.3,23.8,...,0.2,89.4,0.4,88.8,0.3,88.5,0.2,77.3,6.9,star
2,915856,61.4,126.5,24.1,434.2,80.8,2.2,86.1,1.4,90.3,...,0.1,33.4,0.1,30.7,0.1,30.9,0.1,31.5,0.0,star
3,720026,12073.5,20181.5,3666.1,17101718.7,13748.7,69732.8,15392.9,43757.4,13870.7,...,47.0,6240.3,92.9,6072.5,116.9,6095.6,28.4,6039.1,60.9,star
4,1198160,379.8,491.7,-225.7,11729.1,-8.1,68.4,8.5,26.5,51.7,...,0.8,392.1,1.4,395.1,0.5,396.7,0.4,367.7,11.0,star


In [6]:
def getTrain(train_feature):
    train_feature = train_feature.sample(train_feature.shape[0],replace=False)
    testEnd = int(train_feature.shape[0]/3)
    train_test = train_feature.loc[:testEnd,:]
    train_feature = train_feature.loc[testEnd:,:]
    return (train_test, train_feature)

In [7]:
def balanceData(train_feature):
    train_star = train_feature[train_feature.label=='star'].sample(30000, replace=True)
    unknow_star = train_feature[train_feature.label=='unknown'].sample(30000, replace=True)
    galaxy_star = train_feature[train_feature.label=='galaxy'].sample(30000, replace=True)
    qso_star = train_feature[train_feature.label=='qso'].sample(30000, replace=True)
    sampled_train = pd.concat([train_star,unknow_star,galaxy_star, qso_star])
    features = train_feature.columns[1:-1]
    x= sampled_train[features]
    y, label_map = pd.factorize(sampled_train['label'])
    return x, y, label_map

In [8]:
train_t, train_f =  getTrain(train_feature)
x, y, label_map = balanceData(train_f)
t_x, t_y, _ = balanceData(train_t)

In [34]:
def baseline_model(input_dim):
	# create model
	model = Sequential()
	model.add(Dense(200, input_dim=input_dim, activation='relu'))
	model.add(Dense(80, input_dim=input_dim, activation='relu'))
	model.add(Dense(40, input_dim=input_dim, activation='relu'))
	model.add(Dense(4, activation='softmax'))
	# Compile model
	model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
	return model

In [39]:
clf = RandomForestClassifier(n_jobs=4, max_depth=10, n_estimators=200)
clf = XGBClassifier(
 learning_rate =0.1,
 n_estimators=200,
 max_depth=6,
 min_child_weight=1,
 gamma=0,
 subsample=0.8,
 colsample_bytree=0.8,
 objective= 'binary:logistic',
 nthread=4,
 scale_pos_weight=1,
 seed=27)
rs = clf.fit(x, y)
preds_test = clf.predict(t_x)
clf.score(t_x, t_y)

0.83103333333333329

In [32]:
t_y

array([0, 0, 0, ..., 3, 3, 3])

In [36]:
#clf = RandomForestClassifier(n_jobs=4, max_depth=10, n_estimators=200)
clf = baseline_model(x.shape[1])
dummy_y = np_utils.to_categorical(y)
rs = clf.fit(x, dummy_y)
preds_test = clf.predict(t_x)
preds_label = argmax(preds_test,axis=1)


Epoch 1/1
120000/120000 [==============================] - 14s 119us/step - loss: 2.1523 - acc: 0.6986


0.7576583333333333

In [37]:
preds_label = argmax(preds_test,axis=1)
1.0*sum(abs(preds_label == t_y))/t_y.size


0.7576583333333333

In [139]:
preds = clf.predict(test_feature.iloc[:,1:])


In [140]:
sb = submit(preds, test_feature, label_map)
sb.pred.value_counts()

/Users/cql/anaconda2/lib/python2.7/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until


star       84091
unknown    12504
galaxy      2454
qso          951
Name: pred, dtype: int64

In [108]:
sb.pred.value_counts()

star       84431
unknown    12243
galaxy      2283
qso         1043
Name: pred, dtype: int64